# Random Forest

Training:
- Get a random subset of the dataset
- Create a decision tree with this random subset.
- Repeat for as many times as the number of trees that we want.

Testing (given a datapoint we want to test)
- Get the predictions from each tree
- Hold a majority vote

In [ ]:
import numpy as np
np.random.seed(42)
from collections import Counter

In [ ]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None): #value will only be passed to leaf nodes
        # Parameters after ,*, are keyword-arguments only, has to be addressed as value=42, and not (...,42)
        self.feature = feature
        self.threshold = threshold # The split value for the feature
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=10, n_features=None):
        self.min_samples_split = min_samples_split # Minimum number of samples required to split an internal node, prevents overfitting
        self.max_depth = max_depth
        self.n_features = n_features 
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,y)

    # Recursive function to grow the tree
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Check if we have reached the stopping criteria
        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            n_labels == 1):


            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        subset_features = np.random.choice(n_features, self.n_features, replace=False) # Only use a subset of unique features

        # Find the best split
        best_thresh, best_feature = self._best_split(X, y, subset_features)
        
        # Create the left and right subtrees

        left_idxs, right_idxs = self._split(X[:,best_feature],best_thresh)

        if len(left_idxs)  == 0 or len(right_idxs) == 0:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    # Finding the best splits and thresholds 
    def _best_split(self, X, y, subset_features):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feature in subset_features:
            X_column = X[:, feature]
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                # Calculate the information gain
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature
                    split_thresh = threshold
        return split_thresh, split_idx


    def _information_gain(self,y,X_column, threshold):
        # IG = E(parent) - [weighted average] * E(children)

        #parent entropy

        parent_entropy = self._entropy(y)

        #create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if(len(left_idxs) == 0 or len(right_idxs) == 0):
            return 0

        #calculate the avg. weighted entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n)*e_l + (n_r/n)*e_r

        #calculate the IG

        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column<=split_thresh).flatten() # gives the index of the arguments that fulfills the args. flatten() flattens the list of lists into a list
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
    def _entropy(self,y):
        # E = - Sum(p(X)*log_2(p(X)))
        hist = np.bincount(y) #creates a historgram [instances_of_0, instances_of_1, ...]
        ps = hist/len(y)
        return -np.sum([p*np.log(p) for p in ps if p>0]) 

    def _most_common_label(self, y):
        # Return the most common label in y
        counter = Counter(y) # REPLACE
        most_common = counter.most_common(1)[0][0]
        return most_common

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x,node.left)
        return self._traverse_tree(x,node.right)

    
    def predict(self, X):
        return np.array([self._traverse_tree(x,self.root) for x in X])
    
    

In [ ]:
# from sklearn.base import BaseEstimator, ClassifierMixin
# BaseEstimator, ClassifierMixin
class RandomForest():
    def __init__(self, n_trees=200, max_depth=10, min_samples_split=3, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []


    def fit(self, X,y):

        n_tot_features = X.shape[1]
        if self.n_features is None:
            self.n_features = int(np.sqrt(n_tot_features))
        else:
            self.n_features = min(self.n_features, n_tot_features) 

        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                        min_samples_split=self.min_samples_split,
                         n_features=self.n_features)
            X_sample, y_sample =self._bootstrap_samples(X,y)
            tree.fit(X_sample,y_sample)
            self.trees.append(tree)
        return self

    def _bootstrap_samples(self,X,y):
        n_samples, _ = X.shape
        idxs = np.random.choice(n_samples, n_samples, replace=True) # sampling with replacement
        return X[idxs], y[idxs]
    
    def _most_common_label(self, y):
        # Return the most common label in y
        counter = Counter(y) # REPLACE
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees]) # [[pred_1st_sample_by_1st_tree, pred_2st_sample_by_1st_tree],
                                                                        #   [pred_1st_sample_by_2nd_tree],[pred_2nd_sample_by_2nd_tree],
                                                                        #   [...], ...]
        #[[all predictions of first sample], [all predictions of second sample], [...], ...]

        tree_predictions = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_predictions])
        return predictions

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV

data = pd.read_csv('data/GenreClassData_30s.txt', sep='\t')
data["TrackID"] = range(len(data))

# Split the data into training and testing sets
train = data[data['Type'] == 'Train']
test = data[data['Type'] == 'Test']


#all_features = [col for col in data.columns if col not in ['Track ID','TrackID', 'File', 'GenreID', 'Genre', 'Type']]
all_features = [
    'zero_cross_rate_mean','zero_cross_rate_std','rmse_mean','rmse_var',
    'spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var',
    'spectral_rolloff_mean','spectral_rolloff_var','spectral_contrast_mean','spectral_contrast_var',
    'spectral_flatness_mean','spectral_flatness_var',
    'chroma_stft_1_mean','chroma_stft_2_mean','chroma_stft_3_mean','chroma_stft_4_mean',
    'chroma_stft_5_mean','chroma_stft_6_mean','chroma_stft_7_mean','chroma_stft_8_mean',
    'chroma_stft_9_mean','chroma_stft_10_mean','chroma_stft_11_mean','chroma_stft_12_mean',
    'chroma_stft_1_std','chroma_stft_2_std','chroma_stft_3_std','chroma_stft_4_std',
    'chroma_stft_5_std','chroma_stft_6_std','chroma_stft_7_std','chroma_stft_8_std',
    'chroma_stft_9_std','chroma_stft_10_std','chroma_stft_11_std','chroma_stft_12_std',
    'tempo',
    'mfcc_1_mean','mfcc_2_mean','mfcc_3_mean','mfcc_4_mean','mfcc_5_mean','mfcc_6_mean',
    'mfcc_7_mean','mfcc_8_mean','mfcc_9_mean','mfcc_10_mean','mfcc_11_mean','mfcc_12_mean',
    'mfcc_1_std','mfcc_2_std','mfcc_3_std','mfcc_4_std','mfcc_5_std','mfcc_6_std',
    'mfcc_7_std','mfcc_8_std','mfcc_9_std','mfcc_10_std','mfcc_11_std','mfcc_12_std'
]
'''all_features = [
    'zero_cross_rate_mean','zero_cross_rate_std','rmse_mean','rmse_var',
    'spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var',
    'spectral_rolloff_mean','spectral_rolloff_var','spectral_contrast_mean','spectral_contrast_var',
    'spectral_flatness_mean','spectral_flatness_var',
    'chroma_stft_7_mean',
    
    'tempo',
    'mfcc_1_mean','mfcc_2_mean','mfcc_3_mean','mfcc_4_mean','mfcc_5_mean','mfcc_6_mean',

    'mfcc_2_std','mfcc_3_std','mfcc_4_std','mfcc_5_std', 'mfcc_7_std'
]'''
#Erfaringer: 
#'chroma_stft_x_std' er elendig, drar ned accuracy
features = all_features

#features = ['spectral_rolloff_mean', 'mfcc_1_mean', 'spectral_centroid_mean', 'chroma_stft_10_mean']

targets = ['GenreID'] 

# feature data
X_train, y_train = train[features], train[targets]
X_test, y_test = test[features], test[targets]

X_train_np = X_train.to_numpy()      
y_train_np = y_train.to_numpy().ravel()  # flater ut fra (n,1) til (n,)

X_test_np  = X_test.to_numpy()
y_test_np  = y_test.to_numpy().ravel()



clf = RandomForest()
clf.fit(X_train_np, y_train_np)

#Beste param: {'max_depth': 10, 'min_samples_split': 2, 'n_trees': 150}
#param: {'max_depth': 10, 'min_samples_split': 2, 'n_trees': 200} gave 0.712121
# def __init__(self, n_trees=200, max_depth=10, min_samples_split=3, n_features=None): gave 0.727272
#def __init__(self, n_trees=200, max_depth=10, min_samples_split=4, n_features=None): gave 0.712121



# param_grid = {
#     'n_trees': [150, 200],
#     'max_depth': [10, 15],
#     'min_samples_split': [2, 4],
# }

# grid = GridSearchCV(
#     estimator=RandomForest(),
#     param_grid=param_grid,
#     cv=5,
#     scoring='accuracy',
#     n_jobs=-1
# )

# grid.fit(X_train_np, y_train_np)
# print("Beste param:", grid.best_params_)
# print("Best score:", grid.best_score_)






In [ ]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

predictions = clf.predict(X_test_np)
acc = accuracy(y_test_np, predictions)
print(acc)